In [1]:
import pandas as pd, numpy as np

In [2]:
df_clean=pd.read_csv("cleaned_text.csv")
df_clean.head()

,Unnamed: 0,text,target
0,0,ex wife threatening suiciderecently left wife ...,1
1,1,weird get affected compliment coming someone k...,0
2,2,finally almost never hear bad year ever swear ...,0
3,3,need helpjust help im cry hard,1
4,4,losthello name adam struggling year afraid pas...,1


In [3]:
df_clean.drop("Unnamed: 0",axis=1,inplace=True)
df_clean.head()

,text,target
0,ex wife threatening suiciderecently left wife ...,1
1,weird get affected compliment coming someone k...,0
2,finally almost never hear bad year ever swear ...,0
3,need helpjust help im cry hard,1
4,losthello name adam struggling year afraid pas...,1


In [4]:
df_clean.dropna(inplace=True)

In [5]:
X = df_clean.iloc[:, 0].values
y = df_clean.iloc[:, 1].values

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(max_features=300)
model_tfidf=tfidf.fit_transform(X)

In [8]:
X=model_tfidf.toarray()

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 9852)

### Exhaustively testing different classifiers on this model:

In [6]:
from sklearn.metrics import f1_score

#### Naive Bayes

In [13]:
from sklearn.naive_bayes import MultinomialNB
NB = MultinomialNB()
NB.fit(X_train, y_train)
y_pred = NB.predict(X_test)
f1_score(y_test,y_pred)

0.8354646433667143

#### XGBoost

In [43]:
from sklearn.ensemble import GradientBoostingClassifier
XGB = GradientBoostingClassifier(n_estimators=100,learning_rate=0.3)
XGB.fit(X_train,y_train)
y_pred=XGB.predict(X_test)
f1_score(y_test,y_pred)

0.8871928131264385

#### Random Forest

In [46]:
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(n_estimators=100,min_samples_split=5)
RF.fit(X_train,y_train)
y_pred=RF.predict(X_test)
f1_score(y_test,y_pred)

0.8875116562656912

#### SVM

In [11]:
from sklearn.svm import SVC
SVM=SVC(kernel='linear')
SVM.fit(X_train,y_train)
y_pred=SVM.predict(X_test)
f1_score(y_test,y_pred)

NameError: name 'f1_score' is not defined

In [15]:
from sklearn.svm import SVC
SVM=SVC(kernel='rbf')
SVM.fit(X_train,y_train)
y_pred=SVM.predict(X_test)
f1_score(y_test,y_pred)

0.8943723922370366

#### KNN

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.90, random_state = 9852)

In [15]:
from sklearn.neighbors import KNeighborsClassifier
KNN=KNeighborsClassifier(n_neighbors=7)
KNN.fit(X_train,y_train)
y_pred=KNN.predict(X_test)
f1_score(y_test,y_pred)

0.3272823432420241

#### Neural Networks:

In [6]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Embedding, BatchNormalization, Softmax
from tensorflow.keras.optimizers import SGD

C:\Users\NOVA\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\NOVA\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\NOVA\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\NOVA\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_nn=TfidfVectorizer(max_features=100)
model_tfidf_nn=tfidf_nn.fit_transform(X)

In [8]:
X=model_tfidf_nn.toarray()
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.9, random_state = 9852)

In [9]:
max_words=len(X_train)

In [10]:
opt = SGD(lr=0.5, momentum=0.95)
NN = Sequential()
NN.add(Embedding(20000, 128))
NN.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
NN.add(Dense(1, activation='sigmoid'))
NN.add(Dropout(0.2))
NN.add(Softmax())
NN.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         2560000   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
_________________________________________________________________
dropout (Dropout)            (None, 1)                 0         
_________________________________________________________________
softmax (Softmax)            (None, 1)                 0         
Total params: 2,69

In [11]:
NN.compile(loss="binary_crossentropy",optimizer=opt,metrics=["Accuracy"])

In [71]:

y_train

array(['0', '1', '0', ..., '1', '0', '1'], dtype='<U21')

In [ ]:
NN.fit(X_train,y_train,batch_size=50,epochs=30,verbose=1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/30
23198/23198 [==============================] - 150s 6ms/sample - loss: 7.5397 - accuracy: 0.5083
Epoch 2/30
23198/23198 [==============================] - 164s 7ms/sample - loss: 7.5397 - accuracy: 0.5083
Epoch 3/30
23198/23198 [==============================] - 9740s 420ms/sample - loss: 7.5397 - accuracy: 0.5083
Epoch 4/30
20700/23198 [=========================>....] - ETA: 27s - loss: 7.5370 - accuracy: 0.5085

In [13]:
NN.fit(X_train,y_train,batch_size=200,epochs=5,verbose=1)

Epoch 1/5
23198/23198 [==============================] - 124s 5ms/sample - loss: 7.5397 - accuracy: 0.5083
Epoch 2/5
23198/23198 [==============================] - 116s 5ms/sample - loss: 7.5397 - accuracy: 0.5083
Epoch 3/5
23198/23198 [==============================] - 116s 5ms/sample - loss: 7.5397 - accuracy: 0.5083
Epoch 4/5
23198/23198 [==============================] - 120s 5ms/sample - loss: 7.5397 - accuracy: 0.5083
Epoch 5/5
23198/23198 [==============================] - 136s 6ms/sample - loss: 7.5397 - accuracy: 0.5083
